# Walmart API overview
##### Search API used for look ups. We will use this first
##### Taxonomy API used for classifications.  We want to find a way to count classifications

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import urllib
import json
import sys
import pprint
import os
import random

# We are using 2 api Keys for redundancy and because
# there is a limit to the number of calls
api_key = "4tsbjxvbrwnpcjdsh277csqu"
api_alt = "anrtcd5dheh9758z5wgdy677"

# Base url for all API look ups
base_url = "http://api.walmartlabs.com/v1/" 

# Variables to specify the API
search = "search?"
paginated_products = "paginated/items?"

# Base url for search
search_url = base_url + search
paginated_url = base_url + paginated_products



# Search API

In [3]:
# Load search criteria for Walmart Search API.
# reference URL: https://developer.walmartlabs.com/docs/read/Search_API

format = "json"
categoryId = "3944"
query = "laptop"
numItems = 25 # max 25. 10 is default
start = 1
sort = "price"
order = "desc"
facet = "on" # boolean
min = str(0) # lower end of range for search within price range
max = str(200) # higher end of range price for search with price range
facet_range = (f"price:[{min}%20TO%20{max}]") # note the format [num1:num2] does not work. Use %20TO%20 for ":"



# NOTE: we are using 2 methods to create urls. Because the params argument does not format the output
# in the manner needed to get the API to work.  This is used only for price range lookups

# manual url. Used for price range lookup only
manual_url = (f"{search_url}apiKey={api_alt}&format={format}&categoryId={categoryId}&query={query}&numItems={numItems}&facet={facet}&facet.range={facet_range}")

print(manual_url)

# API parameters for other lookups
params = {
    "apiKey": api_alt,
    #"start": start, # note keep turned off.  We can't get this to work yet
    "format": "json", # json|xml
    "categoryId": categoryId,
    "query": query,
    "numItems": numItems,
    #"sort": sort, #allowed sort types are [relevance, price, title, bestseller, customerRating, new]
    #"order": order,  #allowed values are [asc, desc]
    #"facet":facet, # note keep turned off.  We can't get this to work yet
    #"facet.range":facet_range    # note keep turned off.  We can't get this to work yet
}

http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[0%20TO%20200]


In [12]:
# Loop by pricing parameters to pull out entire data set for a product category
# to save time we are going to hard code some numbers then go back later if we have time

# Number of items found in our search
# there are 20,747 total results for laptop in electronics
# step 0 there are 3089 results in the price range $0-$200
# step 1 there are 3015 results in the price range $201-$400
# step 2 there are 2053 results in the price range $401-$600
# step 3 there are 2995 results in the price range $601-$1000
# step 4 there are 2021 results in the price range $1001-$1400
# step 5 there are 2726 results in the price range $1401-$1800
# step 6 there are 1873 results in the price range $1801-$2200
# step 7 there are 1011 results in the price range $2201-$2600
# step 8 there are 863 results in the price range $2601-3000
# step 9 there are 1096 results in the price range above $3000

category_total = 20747 #hard coded from broad search
step0 = 200*25//3089
step1 = 200*25//3015
step2 = 200*25//2053
step3 = 400*25//2995
step4 = 400*25//2021
step5 = 400*25//2726
step6 = 400*25//1873
step7 = 400*25//1011
step8 = 400*25//863

# create a function to perform the price search

def price_data(lowRange, highRange):
    facet_range = (f"price:[{lowRange}%20TO%20{highRange}]")
    price_url = (f"{search_url}apiKey={api_alt}&format={format}&categoryId={categoryId}&query={query}&numItems={numItems}&facet={facet}&facet.range={facet_range}")
    walmart_data2 = requests.get(price_url)
    walmart_data2 = walmart_data2.json()
    return walmart_data2


#if walmart_data2.status_code==200:


# #    print (price_url
#    try:       
#         a = []
#         if not os.path.isfile('Walmart_datasets/walmart_laptops.json'):
#             a.append(walmart_data2['items'])
#             with open('Walmart_datasets/walmart_laptops.json', mode='w') as f:
#                 f.write(json.dumps(walmart_data2['items'], indent=4))
#         else:
#             with open('Walmart_datasets/walmart_laptops.json') as feedsjson:
#                 feeds = json.load(feedsjson)
            
#             feeds.append(walmart_data2['items'])
#             with open('Walmart_datasets/walmart_laptops.json', mode='w') as f:
#                 f.write(json.dumps(feeds, indent=4))

#     except:
#         print ("Skipped line")
                


In [26]:
# Use for loop to go through prices in the steps defined

searchCount = 0
itemCount = 0
sleepCount = 0
low = 600
high = 700

with open('Walmart_datasets/walmart_laptops_test2.json', mode='w') as f:

    for itemCount in range (1,4):
        searchCount += 1
        itemCount = 25*searchCount
        print(f"search / item count = {searchCount} / {itemCount}")

        # We have had issues pulling all the data.  Walmart could be shutting us down
        # We will add some random sleep every 100 queries to try to trick the servers
        sleepCount += 1
        if sleepCount == 100:
            sleepTime = random.randint(60, 120)
            time.sleep(sleepTime)
            print(f"resting for {sleepTime} seconds")
            sleepCount = 0
        elif high < 200:
            high = low + step0
            data = price_data(low,high)
            f.write(json.dumps(data) + '\n')
            low = high
        elif high < 400:
            high = low + step1
            data = price_data(low,high)
            f.write(json.dumps(data) + '\n')
            low = high
        elif high < 600:
            high = low + step2
            data = price_data(low,high)
            f.write(json.dumps(data) + '\n')
            low = high
        elif high < 1000:
            high = low + step3
            data = price_data(low,high)
            f.write(json.dumps(data) + '\n')
            low = high      
        elif high < 1400:
            high = low + step4
            data = price_data(low,high)
            f.write(json.dumps(data) + '\n')
            low = high
        elif high < 1800:
            high = low + step5
            data = price_data(low,high)
            f.write(json.dumps(data) + '\n')       
            low = high
        elif high < 2200:
            high = low + step6
            data = price_data(low,high)
            f.write(json.dumps(data) + '\n')    
            low = high    
        elif high < 2600:
            high = low + step7
            data = price_data(low,high)
            f.write(json.dumps(data) + '\n')
            low = high
        elif high < 3000:
            high = low + step8
            data = price_data(low,high)
            f.write(json.dumps(data) + '\n')
            low = high
        elif high < 8000:
            high = low + step8
            data = price_data(low,high)
            f.write(json.dumps(data) + '\n')
            low = high
        else:
            print ('finished')
            print (f'item count = {itemCount}')
            print (f'final search count = {searchCount}')
            print (f'final high price is ${high}')
            break
               
      



search / item count = 1 / 25
search / item count = 2 / 50
search / item count = 3 / 75


In [43]:
for i in range(1):
    data = price_data(600, 700)
    data = json.dumps(data)
    j = json.loads(data)
    print(j)

{'query': 'laptop', 'categoryId': '3944', 'sort': 'relevance', 'responseGroup': 'base', 'totalResults': 769, 'start': 1, 'numItems': 25, 'items': [{'itemId': 322570741, 'parentItemId': 322570741, 'name': 'HP Pavilion 15-cs0051wm 15.6 Touchscreen Laptop, Windows 10 Home, Intel Core i5-8250U Processor, 8GB SDRAM Memory, 16GB Intel®Optane Memory, 1TB Hard Drive, Pale Gold', 'salePrice': 639.0, 'upc': '192545383455', 'categoryPath': 'Electronics/Computers/Laptops/Shop Laptops by Type/Touchscreen Laptops', 'shortDescription': "Work. Write. Play. Naturally. A durable 360-degree geared hinge gets you in the perfect position to work, write, watch, and play. With the latest technology and an ultrathin and light design, this powerful PC is the one device for everything you're into. Full HD Micro-edge touch display, Dual HP speakers, HP Audio Boost, and tuning by B PLAY deliver a rich, authentic audio and video experience. All your activities become easier and faster than before with the 8th Gene

In [81]:
from pprint import pprint
import os, sys

array = []

filepath = os.path.join('Walmart_datasets/walmart_laptops_test2.json')
with open(filepath, 'r') as jsonfile:
#     data = jsonfile.read()
    for line in jsonfile:
        data = line.strip()
        json_data = json.loads(data)
        for ob in json_data['items']:
            array.append(ob)

    
#         print(len(json_data['items']))
#         break
#         for item in json_data['items']:
#             print(item['name'])
#     sys.exit()
    
    
#     json_data = json.load(data)
#     print (len(json_data))

    
#     for i in range(len(json_data)):
#         try:
#             print (json_data['itemId'])
#         except:
#             print(f"skipped entry {i}")
    


In [83]:
pd.DataFrame(array)

,addToCartUrl,affiliateAddToCartUrl,availableOnline,bundle,categoryNode,categoryPath,customerRating,customerRatingImage,freeShippingOver35Dollars,giftOptions,...,productTrackingUrl,productUrl,rhid,salePrice,sellerInfo,shortDescription,standardShipRate,stock,thumbnailImage,upc
0,http://c.affil.walmart.com/t/api02?l=http%3A%2...,http://linksynergy.walmart.com/fs-bin/click?id...,True,False,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,NaN,NaN,False,NaN,...,http://linksynergy.walmart.com/fs-bin/click?id...,http://c.affil.walmart.com/t/api02?l=https%3A%...,27371,600.43,"SaveCentral, LLC",&quot;Marketing Information from the Manufactu...,0.00,Available,https://i5.walmartimages.com/asr/05c8c9c6-394b...,NaN
1,http://c.affil.walmart.com/t/api02?l=http%3A%2...,http://linksynergy.walmart.com/fs-bin/click?id...,True,False,3944_3951_1089430,Electronics/Computers/Laptops/Laptops By Brand...,NaN,NaN,NaN,"{'allowGiftWrap': False, 'allowGiftMessage': T...",...,http://linksynergy.walmart.com/fs-bin/click?id...,http://c.affil.walmart.com/t/api02?l=https%3A%...,27371,601.16,NaN,"Ideal for work or play,&nbsp;the Refurbished H...",0.00,Available,https://i5.walmartimages.com/asr/a74f304e-776a...,825633444782
2,http://c.affil.walmart.com/t/api02?l=http%3A%2...,http://linksynergy.walmart.com/fs-bin/click?id...,True,False,3944_3951_1089430,Electronics/Computers/Laptops/Laptops By Brand...,NaN,NaN,False,NaN,...,http://linksynergy.walmart.com/fs-bin/click?id...,http://c.affil.walmart.com/t/api02?l=https%3A%...,27371,602.99,DemProductSales,&quot;Acer Aspire 5 A515-51-596K 15.6&quot;&qu...,0.00,Available,https://i5.walmartimages.com/asr/c51237b8-1ee0...,191114485613
3,http://c.affil.walmart.com/t/api02?l=http%3A%2...,http://linksynergy.walmart.com/fs-bin/click?id...,True,False,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,NaN,NaN,False,NaN,...,http://linksynergy.walmart.com/fs-bin/click?id...,http://c.affil.walmart.com/t/api02?l=https%3A%...,27371,602.49,Tech For Less Inc,&quot;Dell Inspiron 13 5000 I5378-7171GRY 2-in...,0.00,Available,https://i5.walmartimages.com/asr/7d08fdc4-3ec8...,NaN
4,http://c.affil.walmart.com/t/api02?l=http%3A%2...,http://linksynergy.walmart.com/fs-bin/click?id...,True,False,3944_3951_1089430,Electronics/Computers/Laptops/Laptops By Brand...,NaN,NaN,False,NaN,...,http://linksynergy.walmart.com/fs-bin/click?id...,http://c.affil.walmart.com/t/api02?l=https%3A%...,27371,603.00,Big Deal GA,Processor &amp; Memory:8th Gen Intel&reg; Core...,0.00,Available,https://i5.walmartimages.com/asr/34aac4b2-77b6...,680175984918
5,http://c.affil.walmart.com/t/api02?l=http%3A%2...,http://linksynergy.walmart.com/fs-bin/click?id...,True,False,3944_3951_1089430,Electronics/Computers/Laptops/Laptops By Brand...,NaN,NaN,False,NaN,...,http://linksynergy.walmart.com/fs-bin/click?id...,http://c.affil.walmart.com/t/api02?l=https%3A%...,27371,603.00,Big Deal GA,Processor &amp; Memory:8th Gen Intel&reg; Core...,0.00,Available,https://i5.walmartimages.com/asr/de67ea91-56b2...,680175985045
6,http://c.affil.walmart.com/t/api02?l=http%3A%2...,http://linksynergy.walmart.com/fs-bin/click?id...,True,False,3944_3951_1089430,Electronics/Computers/Laptops/Laptops By Brand...,NaN,NaN,False,NaN,...,http://linksynergy.walmart.com/fs-bin/click?id...,http://c.affil.walmart.com/t/api02?l=https%3A%...,27371,600.00,Skytech USA LLC.,Certified Refurbished Lenovo ThinkPad T450S 14...,0.00,Available,https://i5.walmartimages.com/asr/4e72b738-4479...,720089860028
7,http://c.affil.walmart.com/t/api02?l=http%3A%2...,http://linksynergy.walmart.com/fs-bin/click?id...,True,False,3944_3951_1089430,Electronics/Computers/Laptops/Laptops By Brand...,NaN,NaN,False,NaN,...,http://linksynergy.walmart.com/fs-bin/click?id...,http://c.affil.walmart.com/t/api02?l=https%3A%...,27371,602.84,Your EPower,Good working condition! Has some signs of gene...,0.00,Available,https://i5.walmartimages.com/asr/f6628d19-3ce9...,782516147195
8,http://c.affil.walmart.com/t/

In [78]:
for line in array:
    df = pd.DataFrame(line)
    df.head(2)

In [70]:
df = None
one = None
for line in array:
#     print(len(line))
#     print(type(line))
    data = line.strip()
    json_data = json.loads(data)
#     print (len(json_data))
    one = json_data
    
#     df = pd.DataFrame(json_data)
    break
#     for item in json_data['items']:
#         print(item['itemId'])
    

In [73]:
pd.DataFrame(one['items'])

,addToCartUrl,affiliateAddToCartUrl,availableOnline,bundle,categoryNode,categoryPath,freeShippingOver35Dollars,giftOptions,imageEntities,isTwoDayShippingEligible,...,productTrackingUrl,productUrl,rhid,salePrice,sellerInfo,shortDescription,standardShipRate,stock,thumbnailImage,upc
0,http://c.affil.walmart.com/t/api02?l=http%3A%2...,http://linksynergy.walmart.com/fs-bin/click?id...,True,False,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,False,NaN,[{'thumbnailImage': 'https://i5.walmartimages....,NaN,...,http://linksynergy.walmart.com/fs-bin/click?id...,http://c.affil.walmart.com/t/api02?l=https%3A%...,27371,600.43,"SaveCentral, LLC",&quot;Marketing Information from the Manufactu...,0.00,Available,https://i5.walmartimages.com/asr/05c8c9c6-394b...,NaN
1,http://c.affil.walmart.com/t/api02?l=http%3A%2...,http://linksynergy.walmart.com/fs-bin/click?id...,True,False,3944_3951_1089430,Electronics/Computers/Laptops/Laptops By Brand...,NaN,"{'allowGiftWrap': False, 'allowGiftMessage': T...",[{'thumbnailImage': 'https://i5.walmartimages....,False,...,http://linksynergy.walmart.com/fs-bin/click?id...,http://c.affil.walmart.com/t/api02?l=https%3A%...,27371,601.16,NaN,"Ideal for work or play,&nbsp;the Refurbished H...",0.00,Available,https://i5.walmartimages.com/asr/a74f304e-776a...,825633444782
2,http://c.affil.walmart.com/t/api02?l=http%3A%2...,http://linksynergy.walmart.com/fs-bin/click?id...,True,False,3944_3951_1089430,Electronics/Computers/Laptops/Laptops By Brand...,False,NaN,[{'thumbnailImage': 'https://i5.walmartimages....,NaN,...,http://linksynergy.walmart.com/fs-bin/click?id...,http://c.affil.walmart.com/t/api02?l=https%3A%...,27371,602.99,DemProductSales,&quot;Acer Aspire 5 A515-51-596K 15.6&quot;&qu...,0.00,Available,https://i5.walmartimages.com/asr/c51237b8-1ee0...,191114485613
3,http://c.affil.walmart.com/t/api02?l=http%3A%2...,http://linksynergy.walmart.com/fs-bin/click?id...,True,False,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,False,NaN,[{'thumbnailImage': 'https://i5.walmartimages....,NaN,...,http://linksynergy.walmart.com/fs-bin/click?id...,http://c.affil.walmart.com/t/api02?l=https%3A%...,27371,602.49,Tech For Less Inc,&quot;Dell Inspiron 13 5000 I5378-7171GRY 2-in...,0.00,Available,https://i5.walmartimages.com/asr/7d08fdc4-3ec8...,NaN
4,http://c.affil.walmart.com/t/api02?l=http%3A%2...,http://linksynergy.walmart.com/fs-bin/click?id...,True,False,3944_3951_1089430,Electronics/Computers/Laptops/Laptops By Brand...,False,NaN,[{'thumbnailImage': 'https://i5.walmartimages....,NaN,...,http://linksynergy.walmart.com/fs-bin/click?id...,http://c.affil.walmart.com/t/api02?l=https%3A%...,27371,603.00,Big Deal GA,Processor &amp; Memory:8th Gen Intel&reg; Core...,0.00,Available,https://i5.walmartimages.com/asr/34aac4b2-77b6...,680175984918
5,http://c.affil.walmart.com/t/api02?l=http%3A%2...,http://linksynergy.walmart.com/fs-bin/click?id...,True,False,3944_3951_1089430,Electronics/Computers/Laptops/Laptops By Brand...,False,NaN,[{'thumbnailImage': 'https://i5.walmartimages....,NaN,...,http://linksynergy.walmart.com/fs-bin/click?id...,http://c.affil.walmart.com/t/api02?l=https%3A%...,27371,603.00,Big Deal GA,Processor &amp; Memory:8th Gen Intel&reg; Core...,0.00,Available,https://i5.walmartimages.com/asr/de67ea91-56b2...,680175985045
6,http://c.affil.walmart.com/t/api02?l=http%3A%2...,http://linksynergy.walmart.com/fs-bin/click?id...,True,False,3944_3951_1089430,Electronics/Computers/Laptops/Laptops By Brand...,False,NaN,[{'thumbnailImage': 'https://i5.walmartimages....,NaN,...,http://linksynergy.walmart.com/fs-bin/click?id...,http://c.affil.walmart.com/t/api02?l=https%3A%...,27371,600.00,Skytech USA LLC.,Certified Refurbished Lenovo ThinkPad T450S 14...,0.00,Available,https://i5.walmartimages.com/asr/4e72b738-4479...,720089860028
7,http://c.affil.walmart.com/t/api02?l=http%3A%2...,http://linksynergy.walmart.com/fs-bin/click?id...,True,False,3944_3951_1089430,Electronics/Computers/Laptop